##Скачиваем датасет

In [3]:
!wget -P /content  https://raw.githubusercontent.com/julianzim/suppi_trial_task/main/trial_task.json

--2023-09-18 11:21:55--  https://raw.githubusercontent.com/julianzim/suppi_trial_task/main/trial_task.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40594 (40K) [text/plain]
Saving to: ‘/content/trial_task.json’

trial_task.json     100%[===================>]  39.64K  --.-KB/s    in 0.003s  

2023-09-18 11:21:55 (12.6 MB/s) - ‘/content/trial_task.json’ saved [40594/40594]



##Импорты

In [1]:
import pandas as pd
import numpy as np
import json

##Читаем данные

In [ ]:
# with open("trial_task.json", "r") as read_file:
#     data = json.load(read_file)

In [4]:
df = pd.read_json('/content/trial_task.json')
df.head(10)

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."
5,87044,остров невезения,-15,"[{'product': 'плюмбус', 'price': 250, 'quantit..."
6,58598,гиперборея,-160,"[{'product': 'плюмбус', 'price': 250, 'quantit..."
7,5430,гиперборея,-80,"[{'product': 'ломтик июльского неба', 'price':..."
8,60502,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."
9,96473,Мордор,-20,"[{'product': 'зеленая пластинка', 'price': 10,..."


##Задание 1
Найти тариф стоимости доставки для каждого склада

In [5]:
df['highway_price'] = df['highway_cost'] / df.products.apply(lambda x: sum([line['quantity'] for line in x]))
df.head()

,order_id,warehouse_name,highway_cost,products,highway_price
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':...",-10.0
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,...",-15.0
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,...",-25.0
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,...",-10.0
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric...",-25.0


In [6]:
highway_prices = df[['warehouse_name', 'highway_price']].drop_duplicates()
highway_prices

,warehouse_name,highway_price
0,Мордор,-10.0
1,хутор близ Диканьки,-15.0
2,отель Лето,-25.0
5,остров невезения,-5.0
6,гиперборея,-20.0


##Задание 2
Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами `'product', 'quantity', 'income', 'expenses', 'profit'`)

##Задание 3
Составить табличку со столбцами `'order_id'` (id заказа) и `'order_profit'` (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

##Задание 4
Составить табличку типа `'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse'` (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

##Задание 5
Взять предыдущую табличку и отсортировать `'percent_profit_product_of_warehouse'` по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться `'accumulated_percent_profit_product_of_warehouse'`. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца `'percent_profit_product_of_warehouse'`.

##Задание 6
Присвоить A,B,C - категории на основании значения накопленного процента (`'accumulated_percent_profit_product_of_warehouse'`). Если значение накопленного процента меньше или равно 70, то категория A.
Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как `'category'`